In [521]:
using LowLevelFEM, LinearAlgebra

In [522]:
gmsh.initialize()

In [523]:
function cube_mesh(; lx=1.0, ly=1.0, lz=1.0, n=10, dx=lx / n, dy=ly / n, dz=lz / n)

    gmsh.option.setNumber("General.Verbosity", 0)

    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addBox(0.0, 0.0, 0.0, lx, ly, lz)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(2, [1], -1, "left")
    gmsh.model.addPhysicalGroup(2, [2], -1, "right")
    gmsh.model.addPhysicalGroup(2, [6], -1, "front")
    gmsh.model.addPhysicalGroup(2, [5], -1, "rear")
    gmsh.model.addPhysicalGroup(2, [3], -1, "bottom")
    gmsh.model.addPhysicalGroup(2, [4], -1, "top")

    gmsh.model.addPhysicalGroup(3, [1], -1, "volume")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    gmsh.model.mesh.setTransfiniteAutomatic()

    gmsh.option.setNumber("Mesh.ElementOrder", 1)

    # Characteristic length control
    lc = min(dx, dy, dz)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(3)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


cube_mesh (generic function with 1 method)

In [524]:
function square_mesh(; lx=1.0, ly=1.0, n=10, dx=lx / n, dy=ly / n)

    gmsh.option.setNumber("General.Verbosity", 0)

    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addRectangle(0.0, 0.0, 0.0, lx, ly)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(1, [1], -1, "bottom")
    gmsh.model.addPhysicalGroup(1, [2], -1, "right")
    gmsh.model.addPhysicalGroup(1, [3], -1, "top")
    gmsh.model.addPhysicalGroup(1, [4], -1, "left")

    gmsh.model.addPhysicalGroup(2, [1], -1, "surface")

    gmsh.model.addPhysicalGroup(0, [1], -1, "leftbottom")
    gmsh.model.addPhysicalGroup(0, [2], -1, "rightbottom")
    gmsh.model.addPhysicalGroup(0, [3], -1, "righttop")
    gmsh.model.addPhysicalGroup(0, [4], -1, "lefttop")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    #gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    #gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    #gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    d = min(dx, dy)

    gmsh.model.mesh.setTransfiniteCurve(1, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(3, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(2, ceil(ly / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(4, ceil(ly / d) + 1)

    gmsh.model.mesh.setTransfiniteSurface(1)

    gmsh.option.setNumber("Mesh.ElementOrder", 1)

    # Characteristic length control
    #lc = min(dx, dy)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(2)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


square_mesh (generic function with 1 method)

In [525]:
square_mesh(lx=100, ly=40, n=10)

In [526]:
mat = Material("surface")
prob = Problem([mat], type=:PlaneStress)

Problem("", :PlaneStress, 2, 2, Material[Material("surface", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 286, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [527]:
suppL = BoundaryCondition("left", ux=0, uy=0)
suppR = BoundaryCondition("right", uy=0)

load = BoundaryCondition("bottom", fy=-1)

BoundaryCondition("bottom", nothing, nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [528]:
μ = mat.μ
λ = mat.λ

115384.61538461536

In [529]:
K = poissonMatrix(prob, coefficient=2μ) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ)

sparse([1, 2, 9, 10, 139, 140, 141, 142, 1, 2  …  77, 78, 551, 552, 553, 554, 569, 570, 571, 572], [1, 1, 1, 1, 1, 1, 1, 1, 2, 2  …  572, 572, 572, 572, 572, 572, 572, 572, 572, 572], [115384.61538461536, 48076.92307692308, -76923.07692307692, 9615.384615384613, 19230.76923076923, -9615.384615384613, -57692.307692307695, -48076.92307692308, 48076.92307692308, 115384.61538461536  …  48076.92307692308, -57692.307692307666, -48076.92307692308, -57692.307692307666, -3.637978807091713e-12, 38461.53846153831, -2.1827872842550278e-11, -153846.15384615376, -1.1641532182693481e-10, 461538.46153846144], 572, 572)

In [530]:
f = loadVector(prob, [load])

nodal VectorField
[0.0; -2.0; … ; 0.0; 0.0;;]

In [531]:
u = solveField(K, f, support=[suppL, suppR])
showDoFResults(u, name="u")

0

In [532]:
S = solveStress(u)
showElementResults(S, name="σ")

1

In [533]:
A = solveStrain(u)
sene = A ⋅ S / 2
showElementResults(sene, name="sene")

2

In [534]:
mx = maximum(elementsToNodes(sene).a)
coef = mapScalarField(x -> x > 0.005mx ? 1 : 0.00001, sene)

elementwise ScalarField
[[1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;]  …  [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0; 1.0; 1.0;;], [1.0; 1.0e-5; 1.0e-5; 1.0e-5;;]]

In [535]:
showElementResults(coef)

3

In [536]:
K = poissonMatrix(prob, coefficient=2μ * coef) + gradDivMatrix(prob, coefficient=0λ * coef) - curlCurlMatrix(prob, coefficient=μ * coef)

sparse([1, 2, 9, 10, 140, 141, 142, 1, 2, 9  …  76, 77, 78, 551, 552, 554, 569, 570, 571, 572], [1, 1, 1, 1, 1, 1, 1, 2, 2, 2  …  572, 572, 572, 572, 572, 572, 572, 572, 572, 572], [76923.07692307692, 19230.76923076923, -38461.53846153846, -19230.76923076923, 19230.76923076923, -38461.53846153847, -19230.76923076923, 19230.76923076923, 76923.07692307692, 19230.769230769234  …  -54487.403846153786, 19230.769230769234, -38461.538461538454, -19230.769230769234, -38461.538461538454, -1.0913936421275139e-10, -1.0913936421275139e-11, -76923.07692307688, -10683.653846153877, 259615.8653846154], 572, 572)

In [537]:
u2 = solveField(K, f, support=[suppL, suppR])
showDoFResults(u2, name="u2")

4

In [538]:
S2 = solveStress(u2)
showElementResults(S2, name="σ2")

5

In [539]:
A2 = solveStrain(u2)
sene2 = A2 ⋅ S2 / 2
showElementResults(sene2 * coef, name="sene")

6

In [540]:
openPostProcessor()

In [541]:
gmsh.finalize()